# Single Particle performance plots


   * [#$p_{T}$ response](#$p_{T}$-response)


In [1]:
import sys
sys.path.insert(0, '..')
import ROOT
import root_numpy as rnp
import python.l1THistos as histos
import math
import uuid
import pandas as pd

from drawingTools import *

ROOT.enableJSVis()
#ROOT.enableJSVis()

#from drawingTools import *

normalized_histos = list()
    

Welcome to JupyROOT 6.14/02


In [2]:
# %load drawingTools.py
# %load drawingTools.py
import ROOT
import math
import uuid
import pandas as pd

# some useful globals, mainly to deal with ROOT idiosyncrasies
c_idx = 0
p_idx = 0
colors = range(1, 12)
colors = [1, 632-4, 416+1, 600-3, 616+1, 432-3]


marker_styles = [8, 21, 22, 23, 33]
stuff = []
f_idx = 0

ROOT.gStyle.SetOptTitle(False)
ROOT.gStyle.SetPadBottomMargin(0.13)
ROOT.gStyle.SetPadLeftMargin(0.13)
ROOT.gStyle.SetPadRightMargin(0.13)

# ROOT.gStyle.SetCanvasBorderMode(0)
# ROOT.gStyle.SetCanvasColor(0)


def DrawPrelimLabel(canvas):
    canvas.cd()
    tex = ROOT.TLatex()
    global stuff
    stuff.append(tex)
    tex.SetTextSize(0.03)
    tex.DrawLatexNDC(0.13,0.91,"#scale[1.5]{CMS} #scale[1.]{Phase-2 Simulation}")
    tex.DrawLatexNDC(0.69,0.91,"14TeV, 200 PU")

    tex.Draw("same");
    return


# void DrawLumiLabel(TCanvas* c, TString Lumi = "35.9")
# {
#   c->cd();

#   TLatex tex;
#   tex.SetTextSize(0.035);
#   TString toDisplay = Lumi + " fb^{-1} (13 TeV)";
#   tex.DrawLatexNDC(0.66,0.91,toDisplay.Data());
#   tex.Draw("same");

#   return;
# }


def SaveCanvas(canvas, name):
    canvas.cd()
    canvas.SaveAs(name+'.pdf')


# void SaveCanvas(TCanvas* c, TString PlotName = "myPlotName")
# {
#   c->cd();
#   c->SaveAs(PlotName + ".pdf");
#   c->SaveAs(PlotName + ".root");

#   return;
# }



def getText(text, ndc_x, ndc_y):
    global stuff
    rtext = ROOT.TLatex(ndc_x, ndc_y, text)
    stuff.append(rtext)
    rtext.SetNDC(True)
    # rtext.SetTextFont(40)
    rtext.SetTextSize(0.03)
    return rtext


def getLegend(x1=0.7, y1=0.71, x2=0.95, y2=0.85):
    global stuff
    legend = ROOT.TLegend(x1, y1, x2, y2)
    stuff.append(legend)
    legend.SetFillColor(0)
    legend.SetFillStyle(0)
    legend.SetBorderSize(0)
    legend.SetTextSize(0.03)
    return legend


def newCanvas(name=None, title=None, height=600, width=800, xdiv=0, ydiv=0, form=4):
    global c_idx
    if name is None:
        name = 'c_{}'.format(uuid.uuid4().hex[:6])
        c_idx += 1
    if title is None:
        title = name
    # print name, title, width, height
    canvas = ROOT.TCanvas(name, title, width, height)
    if(xdiv*ydiv != 0):
        canvas.Divide(xdiv, ydiv)
    global stuff
    stuff.append(canvas)
    return canvas


def drawAndProfileX(plot2d, miny=None, maxy=None, do_profile=True, options='', text=None):
    global p_idx
    if miny and maxy:
        plot2d.GetYaxis().SetRangeUser(miny, maxy)
    c = newCanvas()
    c.SetGrid(1, 1)
    c.cd()
    plot2d.Draw(options)
    ROOT.gPad.SetGrid(1, 1)
    ROOT.gStyle.SetGridColor(15)

    if do_profile:
        profname = plot2d.GetName()+'_prof_'+str(p_idx)
        p_idx += 1
        firstbin = 1
        lastbin = -1
        prof = plot2d.ProfileX(profname, firstbin, lastbin, 's')
        prof.SetMarkerColor(2)
        prof.SetLineColor(2)
        prof.Draw('same')

    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw('same')

    c.Draw()


def draw(histograms,
         labels,
         options='',
         norm=False,
         logy=False,
         min_y=None,
         max_y=None,
         min_x=None,
         max_x=None,
         text=None,
         y_axis_label=None,
         x_axis_label=None,
         v_lines=None,
         h_lines=None,
         do_stats=False,
         do_profile=False,
         do_ratio=False,
         do_write=False,
         write_name=None,
         legend_position=None):
    global colors
    global marker_styles
    global stuff
    global p_idx

    # 0 - determine kind and # of histos
    n_histos = len(histograms)
    if n_histos == 0:
        print '[draw]**ERROR: empy histo list'
        return

    histo_class = histograms[0].ClassName()

    # 1 - check argument consisntency
    if 'TH2' not in histo_class and do_profile:
        print '[draw]**ERROR: do_profile set for histo of class: {}'.format(histo_class)
        return

    # TH1 histograms are overlayed
    # TH2 histograms are drawn side by side in the same canvas
    overlay = True
    if n_histos > 1:
        if 'TH2' in histo_class or 'TH3' in histo_class:
            overlay = False

    # 2 - determine canvas properties
    xdiv = 0
    ydiv = 0
    c_width = 800
    c_height = 600
    # do_ratio = False
    if do_write:
        c_width = 600

    if do_ratio:
        c_height = 800

    if not overlay:
        xdiv = 2
        c_width = 1000.
        ydiv = math.ceil(float(n_histos)/2)
        c_height = 500*ydiv
    canvas = newCanvas(name=None,
                       title=None,
                       height=int(c_height),
                       width=int(c_width),
                       xdiv=int(xdiv),
                       ydiv=int(ydiv))
    if overlay:
        if not do_write:
            canvas.SetRightMargin(0.30)


    canvas.cd()
    leg = getLegend()
    if do_write and legend_position is None:
        leg = getLegend(0.6, 0.7, 0.86, 0.8)
    elif legend_position is not None:
        leg = getLegend(legend_position[0], legend_position[1], legend_position[2], legend_position[3])

    drawn_histos = []
    # drawn_histos = histograms
    for hidx, hist in enumerate(histograms):
        opt = options
        if 'TGraph' not in histo_class:
            hist.SetStats(do_stats)
            hist.SetTitle("")
        else:
            opt = 'P'+options
            hist.SetMarkerSize(0.5)
            hist.SetMarkerStyle(marker_styles[hidx])
        if overlay:
            hist.SetLineColor(colors[hidx])
            if 'TGraph' in histo_class:
                hist.SetMarkerColor(colors[hidx])
                hist.SetMarkerSize(0.5)
                hist.SetMarkerStyle(marker_styles[hidx])

            if hidx:
                opt = 'same,'+opt
            else:
                if 'TGraph' in histo_class:
                    opt = opt+'A'
        else:
            canvas.cd(hidx+1)
        # print hidx, opt
        d_hist = hist
        if norm:
            d_hist = hist.DrawNormalized(opt, 1.)
        else:
            hist.Draw(opt)

        if do_profile:
            profname = d_hist.GetName()+'_prof_'+str(p_idx)
            p_idx += 1
            firstbin = 1
            lastbin = -1
            prof = d_hist.ProfileX(profname, firstbin, lastbin, 's')
            prof.SetMarkerColor(2)
            prof.SetLineColor(2)
            prof.Draw('same')

        if overlay:
            if 'TGraph' not in histo_class:
                leg.AddEntry(hist, labels[hidx], 'l')
            else:
                leg.AddEntry(hist, labels[hidx], 'P')
        else:
            if text:
                newtext = '{}: {}'.format(labels[hidx], text)
                rtext = getText(newtext, 0.15, 0.85)
                rtext.Draw('same')
        drawn_histos.append(d_hist)

    if do_ratio:
        h_ratio = ROOT.TRatioPlot(drawn_histos[0], drawn_histos[1])
        stuff.append(h_ratio)
        h_ratio.Draw()

        pad = canvas.cd(1)
        h_ratio.GetUpperPad().SetBottomMargin(0)
        h_ratio.GetUpperPad().SetRightMargin(0.3)
        if logy:
            h_ratio.GetUpperPad().SetLogy()
        pad.Update()
        pad = canvas.cd(2)
        h_ratio.GetLowerPad().SetTopMargin(0)
        h_ratio.GetLowerPad().SetRightMargin(0.3)
        pad.Update()
        # hratio.GetLowerRefGraph().GetYaxis().SetRangeUser(-6, 6)

    canvas.Update()

    # we now set the axis properties
    max_value = max_y
    min_value = min_y

    if min_y is None:
        min_value = min([hist.GetBinContent(hist.GetMinimumBin()) for hist in drawn_histos])
    if max_y is None:
        max_value = max([hist.GetBinContent(hist.GetMaximumBin()) for hist in drawn_histos])*1.2
    # print min_value, max_value

    for hist in drawn_histos:
        hist.GetXaxis().SetTitleOffset(1.4)
        hist.GetYaxis().SetRangeUser(min_value, max_value)
        if y_axis_label:
            hist.GetYaxis().SetTitle(y_axis_label)
        if x_axis_label:
            hist.GetXaxis().SetTitle(x_axis_label)
        if min_x and max_x:
            hist.GetXaxis().SetRangeUser(min_x, max_x)


    canvas.Draw()

    # we draw additional stuff if needed
    if overlay:
        canvas.cd()
        if len(labels) > 1:
            leg.Draw()
        if text:
            rtext = getText(text, 0.15, 0.85)
            rtext.Draw("same")

    pad_range = range(0, 1)
    if not overlay:
        pad_range = range(1, len(histograms)+1)

    for pad_id in pad_range:
        canvas.cd(pad_id)
        if v_lines:
            for v_line_x in v_lines:
                aline = ROOT.TLine(v_line_x, ROOT.gPad.GetUymin(), v_line_x,  ROOT.gPad.GetUymax())
                aline.SetLineStyle(2)
                aline.Draw("same")
                stuff.append(aline)
        if h_lines:
            for h_line_y in h_lines:
                aline = ROOT.TLine(ROOT.gPad.GetUxmin(), h_line_y, ROOT.gPad.GetUxmax(),  h_line_y)
                aline.SetLineStyle(2)
                aline.Draw("same")
                stuff.append(aline)
        if logy:
            if not do_ratio:
                ROOT.gPad.SetLogy()

        ROOT.gPad.Update()



    canvas.Update()
    if(do_write):
        DrawPrelimLabel(canvas)

    canvas.Draw()
    if(do_write):
        SaveCanvas(canvas, write_name)

    return canvas


files = {}
file_keys = {}


class RootFile:
    def __init__(self, file_name):
        global file
        self.file_name = file_name
        if self.file_name not in files.keys():
            print 'get file: {}'.format(self.file_name)
            files[self.file_name] = ROOT.TFile(self.file_name)
        self._file = files[self.file_name]
        self._file_keys = None

    def cd(self):
        self._file.cd()

    def GetListOfKeys(self):
        global file_keys
        if self.file_name not in file_keys.keys():
            print 'get list'
            file_keys[self.file_name] = self._file.GetListOfKeys()
        self._file_keys = file_keys[self.file_name]
        return self._file_keys


class Sample():
    def __init__(self, name, label, version=None, type=None):
        self.name = name
        self.label = label
        if version:
            version = '_'+version
        else:
            version = ''
        self.histo_filename = '../plots1/histos_{}{}.root'.format(self.name, version)
        self.histo_file = None
        self.type = type

    def __repr__(self):
        return '<{} {}, {}>'.format(self.__class__.__name__, self.histo_filename, self.type)

    def open_file(self):
        self.histo_file = ROOT.TFile(self.histo_filename, 'r')

# sample_names = ['ele_flat2to100_PU0',
#                 'ele_flat2to100_PU200',
#                 'photonPt35_PU0',
#                 'photonPt35_PU200']


class HProxy:
    def __init__(self, classtype, tp, tp_sel, gen_sel, root_file):
        self.classtype = classtype
        self.tp = tp
        self.tp_sel = tp_sel
        self.gen_sel = gen_sel
        self.root_file = root_file
        self.instance = None

    def get(self, debug=False):
        if self.instance is None:
            name = '{}_{}_{}'.format(self.tp, self.tp_sel, self.gen_sel)
            if self.gen_sel is None:
                name = '{}_{}'.format(self.tp, self.tp_sel)
            if debug:
                print '-- HProxy:Get: {}'.format(name)
            self.instance = self.classtype(name, self.root_file, debug)
        return self.instance


class HPlot:
    def __init__(self, samples, tp_sets, tp_selections, gen_selections):
        self.tp_sets = tp_sets
        # self.tp_selections = tp_selections
        # self.gen_selections = gen_selections
        self.pus = []
        self.labels_dict = {}

        for sample in samples:
            self.pus.append(sample.label)
            self.labels_dict[sample.type] = sample.type

        self.data = pd.DataFrame(columns=['sample', 'pu', 'tp', 'tp_sel', 'gen_sel', 'classtype', 'histo'])

        self.labels_dict.update(tp_sets)
        self.labels_dict.update(tp_selections)
        self.labels_dict.update(gen_selections)
        self.labels_dict.update({'PU0': 'PU0', 'PU200': 'PU200'})

    def cache_histo(self,
                    classtype,
                    samples,
                    pus,
                    tps,
                    tp_sels,
                    gen_sels):
        if gen_sels is None:
            gen_sels = [None]

        for sample in samples:
            print sample
            for tp in tps:
                for tp_sel in tp_sels:
                    for gen_sel in gen_sels:
                        print sample, tp, tp_sel, gen_sel
                        self.data = self.data.append({'sample': sample.type,
                                                      'pu': sample.label,
                                                      'tp': tp,
                                                      'tp_sel': tp_sel,
                                                      'gen_sel': gen_sel,
                                                      'classtype': classtype,
                                                      'histo': HProxy(classtype, tp, tp_sel, gen_sel, sample.histo_file)},
                                                     ignore_index=True)

    def get_histo(self,
                  classtype,
                  sample=None,
                  pu=None,
                  tp=None,
                  tp_sel=None,
                  gen_sel=None,
                  debug=False):
        histo = None
        labels = []
        text = ''

        query = '(pu == @pu) & (tp == @tp) & (tp_sel == @tp_sel) & (classtype == @classtype)'
        if gen_sel is not None:
            query += ' & (gen_sel == @gen_sel)'
        else:
            query += ' & (gen_sel.isnull())'
        if sample is not None:
            query += '& (sample == @sample)'

        histo_df = self.data.query(query)

        if histo_df.empty:
            print 'No match found for: pu: {}, tp: {}, tp_sel: {}, gen_sel: {}, classtype: {}'.format(pu, tp, tp_sel, gen_sel, classtype)
            return None, None, None
        if debug:
            print histo_df

        field_counts = histo_df.apply(lambda x: len(x.unique()))
        label_fields = []
        text_fields = []
        # print field_counts
        for field in field_counts.iteritems():
            if(field[1] > 1 and field[0] != 'histo'):
                label_fields.append(field[0])
            if(field[1] == 1 and field[0] != 'histo' and field[0] != 'classtype' and field[0] != 'sample'):
                if(gen_sel is None and field[0] == 'gen_sel'):
                    continue
                text_fields.append(field[0])
#         print 'label fields: {}'.format(label_fields)
#         print 'text fields: {}'.format(text_fields)

        for item in histo_df[label_fields].iterrows():
            labels.append(', '.join([self.labels_dict[tx] for tx in item[1].values if self.labels_dict[tx] != '']))

        # print labels
        text = ', '.join([self.labels_dict[fl] for fl in histo_df[text_fields].iloc[0].values if self.labels_dict[fl] != ''])
        histo = [his.get(debug) for his in histo_df['histo'].values]
        return histo, labels, text


In [3]:
# %load samples.py
# %load samples.py
import ROOT
import pandas as pd
import python.selections as selections

version = 'v87t'
version_V8 = version
version_V9 = 'v115'
version_V10 = 'v102'


# -------------------------------------------------------------------------

samples_ele = [
    Sample('ele_flat2to100_PU0', 'PU0', version, 'ele'),
    Sample('ele_flat2to100_PU200', 'PU200', version, 'ele')
    ]

samples_ele_V8 = [
    Sample('ele_flat2to100_PU0', 'PU0', version_V8, 'V8'),
    Sample('ele_flat2to100_PU200', 'PU200', version_V8, 'V8')
    ]


samples_ele_V9 = [
    Sample('ele_flat2to100_PU0_eg', 'PU0', version_V9, 'ele-V9'),
    Sample('ele_flat2to100_PU200_eg', 'PU200', version_V9, 'ele-V9')
    ]

samples_ele_V10 = [
    Sample('ele_flat2to100_PU0_eg', 'PU0', version_V10, 'ele-V10'),
    Sample('ele_flat2to100_PU200_eg', 'PU200', version_V10, 'ele-V10')
    ]

samples_photons = [
    Sample('photon_flat8to150_PU0_eg', 'PU0', version, 'photon'),
    Sample('photon_flat8to150_PU200_eg', 'PU200', version, 'photon')
    ]

samples_photons_V8 = [
    Sample('photon_flat8to150_PU0_eg', 'PU0', version_V8, 'photon'),
    Sample('photon_flat8to150_PU200_eg', 'PU200', version_V8, 'photon')
    ]

samples_photons_V9 = [
    Sample('photon_flat8to150_PU0_eg', 'PU0', version_V9, 'photons-V9'),
    Sample('photon_flat8to150_PU200_eg', 'PU200', version_V9, 'photons-V9')
    ]

samples_pions = [
    Sample('pion_flat2to100_PU0', 'PU0', version, 'pions'),
    ]

samples_nugus = [
    Sample('nugun_alleta_pu0', 'PU0', version, 'mb'),
    Sample('nugun_alleta_pu200', 'PU200', version, 'mb')
    ]

samples_nugunrates = [
    Sample('nugun_alleta_pu200', 'PU200', version, 'mb')
    ]

samples_nugunrates_V8 = [
    Sample('nugun_alleta_pu200', 'PU200', version, 'V8')
    ]

samples_nugunrates_V9 = [
    Sample('nugun_alleta_pu200', 'PU200', version, 'V9')
    ]


gen_part_selections: 11


In [4]:
# %load settings.py
# === samples =====================================================
import python.selections as selections
import python.collections as collections

import pprint


samples = []

# samples += samples_nugunrates
# samples += samples_nugunrates_V8
samples += samples_ele_V9

for smp in samples:
    smp.open_file()


sample = 'ele-V9'

do_rate = False
do_eff = False
do_reso = True

# === TP ==========================================================
tps = [
       'EG',
       'TkEle',
       'TkEleEL',
       'TkEleBRL',
       'TkEleELBRL',
       'TkEleALL',
       'TkEleELALL'
]

# === Load the Histo Primitives ====================================
histo_primitives = pd.DataFrame()

if do_rate:
    from python.plotters import rate_plotters, eg_rate_plotters

    for plotter in rate_plotters:
        histo_primitives = histo_primitives.append(plotter.get_histo_primitives(), ignore_index=True)
    for plotter in eg_rate_plotters:
        histo_primitives = histo_primitives.append(plotter.get_histo_primitives(), ignore_index=True)
if do_eff or do_reso:
    from python.plotters import eg_genmatched_plotters, track_genmatched_plotters
    for plotter in eg_genmatched_plotters:
        histo_primitives = histo_primitives.append(plotter.get_histo_primitives(), ignore_index=True)
    for plotter in track_genmatched_plotters:
        histo_primitives = histo_primitives.append(plotter.get_histo_primitives(), ignore_index=True)


# print histo_primitives.data.unique()
# === TP selections ================================================
tp_select = {}

for tp in tps:
    tp_select[tp] = histo_primitives[histo_primitives.data == tp].data_sel.unique().tolist()

# ==== GEN selections ===============================================
gen_select ={}
for tp in tps:
    gen_select[tp] = histo_primitives[histo_primitives.data == tp].gen_sel.unique().tolist()

#  ==== labels ===============================================
tp_labels = histo_primitives[['data', 'data_label']].drop_duplicates().set_index('data').T.to_dict('records')[0]
tp_selection_labels = histo_primitives[['data_sel', 'data_sel_label']].drop_duplicates().set_index('data_sel').T.to_dict('records')[0]
gen_selection_labels = histo_primitives[['gen_sel', 'gen_sel_label']].drop_duplicates().set_index('gen_sel').T.to_dict('records')[0]



import pprint
pp = pprint.PrettyPrinter(indent=4)
print '--- TPs: '
pp.pprint(tps)
print '--- TP selections:'
pp.pprint(tp_select)
print '--- GEN selections:'
pp.pprint(gen_select)


# print selections.eg_rate_selections


[EventManager] registering collection: MC
[EventManager] registering collection: GEN
[EventManager] registering collection: TC
[EventManager] registering collection: TCTrue
[EventManager] registering collection: DEF2D
[EventManager] registering collection: DEF2DTrue
[EventManager] registering collection: HMvDRTrue
[EventManager] registering collection: DEF
[EventManager] registering collection: DEFNC
[EventManager] registering collection: HMvDR
[EventManager] registering collection: HMvDREmInt
[EventManager] registering collection: HMvDREmIntMerged
[EventManager] registering collection: HMvDRRebin
[EventManager] registering collection: HMvDRsTC
[EventManager] registering collection: HMvDRNC0
[EventManager] registering collection: HMvDRNC1
[EventManager] registering collection: DEFMerged
[EventManager] registering collection: DEFCalib
[EventManager] registering collection: HMvDRMerged
[EventManager] registering collection: HMvDRfixed
[EventManager] registering collection: HMvDRcylind10


/Users/cerminar/.virtualenvs/HGCTPGPerformance/lib/python2.7/site-packages/ipykernel_launcher.py:68: UserWarning: DataFrame columns are not unique, some columns will be omitted.


In [5]:
tp_labels

{'EG': 'EG',
 'EGALL': 'EG all',
 'EGBRL': 'EG barrel',
 'L1Trk': 'L1Track',
 'L1TrkEmu': 'L1Track EMU',
 'TkEle': 'TkEle',
 'TkEleALL': 'TkEle all',
 'TkEleBRL': 'TkEle barrel',
 'TkEleEL': 'TkEle Ell. match',
 'TkEleELALL': 'TkEle Ell. match all',
 'TkEleELBRL': 'TkEle Ell. match barrel'}

In [6]:
tp_selection_labels

{'EGq2': 'hwQual 2',
 'EGq2Iso0p1': 'hwQual 2, Iso0p1',
 'EGq2Iso0p2': 'hwQual 2, Iso0p2',
 'EGq2Iso0p3': 'hwQual 2, Iso0p3',
 'EGq2Pt10': 'hwQual 2, p_{T}^{L1}>=10GeV',
 'EGq2Pt15': 'hwQual 2, p_{T}^{L1}>=15GeV',
 'EGq2Pt20': 'hwQual 2, p_{T}^{L1}>=20GeV',
 'EGq2Pt25': 'hwQual 2, p_{T}^{L1}>=25GeV',
 'EGq2Pt30': 'hwQual 2, p_{T}^{L1}>=30GeV',
 'EGq2Pt40': 'hwQual 2, p_{T}^{L1}>=40GeV',
 'EGq3': 'hwQual 3',
 'EGq3Iso0p1': 'hwQual 3, Iso0p1',
 'EGq3Iso0p2': 'hwQual 3, Iso0p2',
 'EGq3Iso0p3': 'hwQual 3, Iso0p3',
 'EGq3Pt10': 'hwQual 3, p_{T}^{L1}>=10GeV',
 'EGq3Pt15': 'hwQual 3, p_{T}^{L1}>=15GeV',
 'EGq3Pt20': 'hwQual 3, p_{T}^{L1}>=20GeV',
 'EGq3Pt25': 'hwQual 3, p_{T}^{L1}>=25GeV',
 'EGq3Pt30': 'hwQual 3, p_{T}^{L1}>=30GeV',
 'EGq3Pt40': 'hwQual 3, p_{T}^{L1}>=40GeV',
 'EGq4': 'hwQual 4',
 'EGq4Iso0p1': 'hwQual 4, Iso0p1',
 'EGq4Iso0p2': 'hwQual 4, Iso0p2',
 'EGq4Iso0p3': 'hwQual 4, Iso0p3',
 'EGq4Pt10': 'hwQual 4, p_{T}^{L1}>=10GeV',
 'EGq4Pt15': 'hwQual 4, p_{T}^{L1}>=15GeV',
 'EGq4

In [7]:
gen_selection_labels

{'GEN': '',
 'GENEtaBC': '1.52 < |#eta^{GEN}| <= 2.4',
 'GENEtaBCD': '1.52 < |#eta^{GEN}| <= 2.8',
 'GENEtaBCDPt15': '1.52 < |#eta^{GEN}| <= 2.8, p_{T}^{GEN}>=15GeV',
 'GENEtaBCPt15': '1.52 < |#eta^{GEN}| <= 2.4, p_{T}^{GEN}>=15GeV',
 'GENEtaD': '2.4 < |#eta^{GEN}| <= 2.8',
 'GENEtaDPt15': '2.4 < |#eta^{GEN}| <= 2.8, p_{T}^{GEN}>=15GeV',
 'GENEtaF': '|#eta^{GEN}| <= 1.479',
 'GENPt15': 'p_{T}^{GEN}>=15GeV',
 'GENPt30': 'p_{T}^{GEN}>=30GeV',
 'GENPt35': 'p_{T}^{GEN}>=35GeV',
 'GENPt40': 'p_{T}^{GEN}>=40GeV'}

## Resolution Studies


## Energy resolution and pt response curves

In [8]:
files

{}

In [9]:
%%time


hplot = HPlot(samples, tp_labels, tp_selection_labels, gen_selection_labels)


for tp in tps:
    print tp
    hplot.cache_histo(classtype=histos.EGResoHistos, 
                      samples=samples,
                      pus=[],
                      tps=[tp],
                      tp_sels=histo_primitives[histo_primitives.data == tp].data_sel.unique(),
                      gen_sels=gen_select[tp])




print dir(samples[0])
print samples[0].histo_filename
# for sample in samples:
#     print sample
#     for tp in tpsets:
#         for tp_sel in tpset_selections:
#             for gen_sel in gen_selections:
#                 print sample, tp, tp_sel, gen_sel
#                 hplot.data = hplot.data.append({'sample': 'ele',
#                                                 'pu': sample.label,
#                                                 'tp': tp,
#                                                 'tp_sel': tp_sel,
#                                                 'gen_sel': gen_sel,
#                                                 'classtype': histos.HistoSetReso,
#                                                 'histo': histos.HistoSetReso('{}_{}_{}'.format(tp, tp_sel, gen_sel), sample.histo_file)}
#                                               , ignore_index=True)


EG
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9>
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> EG EGq2 GEN
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> EG EGq2 GENPt15
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> EG EGq2 GENPt30
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> EG EGq2 GENPt35
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> EG EGq2 GENPt40
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> EG EGq2 GENEtaD
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> EG EGq2 GENEtaDPt15
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> EG EGq2 GENEtaBC
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> EG EGq2 GENEtaBCPt15
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> EG EGq2 GENEtaBCD
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> EG EGq2 GENEtaBCDPt15
<Sample ../plot

<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> EG EGq3Pt15 GENPt35
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> EG EGq3Pt15 GENPt40
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> EG EGq3Pt15 GENEtaD
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> EG EGq3Pt15 GENEtaDPt15
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> EG EGq3Pt15 GENEtaBC
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> EG EGq3Pt15 GENEtaBCPt15
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> EG EGq3Pt15 GENEtaBCD
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> EG EGq3Pt15 GENEtaBCDPt15
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> EG EGq3Pt20 GEN
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> EG EGq3Pt20 GENPt15
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> EG EGq3Pt20 GENPt30
<Sample ../plots1/histos_ele_flat2to100_

<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> EG EGq4Pt25 GENEtaBC
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> EG EGq4Pt25 GENEtaBCPt15
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> EG EGq4Pt25 GENEtaBCD
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> EG EGq4Pt25 GENEtaBCDPt15
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> EG EGq4Pt30 GEN
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> EG EGq4Pt30 GENPt15
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> EG EGq4Pt30 GENPt30
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> EG EGq4Pt30 GENPt35
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> EG EGq4Pt30 GENPt40
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> EG EGq4Pt30 GENEtaD
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> EG EGq4Pt30 GENEtaDPt15
<Sample ../plots1/histos_ele_flat2to100_

<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> EG EGq5Pt40 GENPt40
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> EG EGq5Pt40 GENEtaD
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> EG EGq5Pt40 GENEtaDPt15
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> EG EGq5Pt40 GENEtaBC
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> EG EGq5Pt40 GENEtaBCPt15
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> EG EGq5Pt40 GENEtaBCD
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> EG EGq5Pt40 GENEtaBCDPt15
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9>
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9> EG EGq2 GEN
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9> EG EGq2 GENPt15
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9> EG EGq2 GENPt30
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-

<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9> EG EGq3Pt10 GENPt35
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9> EG EGq3Pt10 GENPt40
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9> EG EGq3Pt10 GENEtaD
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9> EG EGq3Pt10 GENEtaDPt15
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9> EG EGq3Pt10 GENEtaBC
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9> EG EGq3Pt10 GENEtaBCPt15
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9> EG EGq3Pt10 GENEtaBCD
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9> EG EGq3Pt10 GENEtaBCDPt15
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9> EG EGq3Pt15 GEN
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9> EG EGq3Pt15 GENPt15
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9> EG EGq3Pt15 GENPt30
<Sample ../plots1/

...

<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9> TkEleALL Pt20 GENEtaBC
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9> TkEleALL Pt20 GENEtaBCD
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9> TkEleALL Pt20LooseTkID GEN
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9> TkEleALL Pt20LooseTkID GENPt15
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9> TkEleALL Pt20LooseTkID GENPt30
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9> TkEleALL Pt20LooseTkID GENPt35
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9> TkEleALL Pt20LooseTkID GENPt40
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9> TkEleALL Pt20LooseTkID GENEtaF
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9> TkEleALL Pt20LooseTkID GENEtaD
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9> TkEleALL Pt20LooseTkID GENEtaBC
<Sample ../plots1/histos_ele

<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> TkEleELALL all GENPt40
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> TkEleELALL all GENEtaF
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> TkEleELALL all GENEtaD
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> TkEleELALL all GENEtaBC
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> TkEleELALL all GENEtaBCD
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> TkEleELALL LooseTkID GEN
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> TkEleELALL LooseTkID GENPt15
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> TkEleELALL LooseTkID GENPt30
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> TkEleELALL LooseTkID GENPt35
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> TkEleELALL LooseTkID GENPt40
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> TkEleELALL Loos

<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> TkEleELALL Pt20Iso0p1 GENEtaBCD
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> TkEleELALL Pt25 GEN
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> TkEleELALL Pt25 GENPt15
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> TkEleELALL Pt25 GENPt30
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> TkEleELALL Pt25 GENPt35
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> TkEleELALL Pt25 GENPt40
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> TkEleELALL Pt25 GENEtaF
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> TkEleELALL Pt25 GENEtaD
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> TkEleELALL Pt25 GENEtaBC
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> TkEleELALL Pt25 GENEtaBCD
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v115.root, ele-V9> TkEleELALL Pt25LooseTkID GE

<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9> TkEleELALL Iso0p1 GENPt30
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9> TkEleELALL Iso0p1 GENPt35
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9> TkEleELALL Iso0p1 GENPt40
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9> TkEleELALL Iso0p1 GENEtaF
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9> TkEleELALL Iso0p1 GENEtaD
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9> TkEleELALL Iso0p1 GENEtaBC
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9> TkEleELALL Iso0p1 GENEtaBCD
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9> TkEleELALL Pt10 GEN
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9> TkEleELALL Pt10 GENPt15
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9> TkEleELALL Pt10 GENPt30
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9>

<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9> TkEleELALL Pt25LooseTkID GENEtaD
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9> TkEleELALL Pt25LooseTkID GENEtaBC
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9> TkEleELALL Pt25LooseTkID GENEtaBCD
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9> TkEleELALL Pt25Iso0p1 GEN
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9> TkEleELALL Pt25Iso0p1 GENPt15
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9> TkEleELALL Pt25Iso0p1 GENPt30
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9> TkEleELALL Pt25Iso0p1 GENPt35
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9> TkEleELALL Pt25Iso0p1 GENPt40
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9> TkEleELALL Pt25Iso0p1 GENEtaF
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v115.root, ele-V9> TkEleELALL Pt25Iso0p1 GENEtaD
<Sample ../p

In [10]:
hplot.data[:10]

,sample,pu,tp,tp_sel,gen_sel,classtype,histo
0,ele-V9,PU0,EG,EGq2,GEN,python.l1THistos.EGResoHistos,<__main__.HProxy instance at 0x117ff6ef0>
1,ele-V9,PU0,EG,EGq2,GENPt15,python.l1THistos.EGResoHistos,<__main__.HProxy instance at 0x118068ea8>
2,ele-V9,PU0,EG,EGq2,GENPt30,python.l1THistos.EGResoHistos,<__main__.HProxy instance at 0x118001a70>
3,ele-V9,PU0,EG,EGq2,GENPt35,python.l1THistos.EGResoHistos,<__main__.HProxy instance at 0x1190e7998>
4,ele-V9,PU0,EG,EGq2,GENPt40,python.l1THistos.EGResoHistos,<__main__.HProxy instance at 0x118071098>
5,ele-V9,PU0,EG,EGq2,GENEtaD,python.l1THistos.EGResoHistos,<__main__.HProxy instance at 0x1049f4878>
6,ele-V9,PU0,EG,EGq2,GENEtaDPt15,python.l1THistos.EGResoHistos,<__main__.HProxy instance at 0x1046ad950>
7,ele-V9,PU0,EG,EGq2,GENEtaBC,python.l1THistos.EGResoHistos,<__main__.HProxy instance at 0x1191edef0>
8,ele-V9,PU0,EG,EGq2,GENEtaBCPt15,python.l1THistos.EGResoHistos,<__main__.HProxy instance at 0x118071248>
9,ele-V9,PU0,EG,EGq2,GENEtaBCD,python.l1THistos.EGResoHistos,<__main__.HProxy instance at 0x118001320>


In [ ]:
hplot.labels_dict

In [ ]:
hfd = hplot.get_histo(histos.EGResoHistos, 'ele-V9', ['PU0'], ['EG'], 'EGq3', 'GEN', True)

print hfd[0]
print hfd[1]
print hfd[2]
print dir(hfd[0][0])

# print hfd[0][0].h_ptRespVeta_mdn()

In [ ]:
print hfd[2]


In [ ]:
def gaussfit(name, project_hist):
    
    max_bin = project_hist.GetMaximumBin()
    max_value = project_hist.GetBinCenter(max_bin)
    rms_value = project_hist.GetRMS()
    max_y = project_hist.GetMaximum()

#     print max_bin, max_value, rms_value
    def gausstail(x, p):
        #         // [Constant] * ROOT::Math::crystalball_function(x, [Alpha], [N], [Sigma], [Mean])
        return p[0] * ROOT.Math.crystalball_function(x[0], p[3], p[4], p[2], p[1])

    fitf = ROOT.TF1('gausstail', "gaus", -100, 100, 5)
#     fitf.SetParNames('norm', 'mean', 'sigma', 'alpha', 'n')
#     fitf.FixParameter(0, 1.)
    fitf.SetParLimits(1, max_value-15.,max_value+15.)

    fitf.SetParameters(max_y,max_value, rms_value)
    draw([project_hist], labels=['fit'], text=name)
    stuff.append(fitf)
    project_hist.Draw("same")
#     c.Draw()
    print '   name: {}, int = {}, max_value = {}, RMS = {}'.format(name, project_hist.Integral(), max_value, rms_value)
    result = project_hist.Fit('gausstail','QERLS+')
    result.Print()
    print '   norm = {}, reso_mean = {}, reso_sigma = {}'.format(result.GetParams()[0], result.GetParams()[1], result.GetParams()[2])
#     func = project_hist.GetFunction("gaus")
    # print '   NDF = {}, chi2 = {}, prob = {}'.format(fitf.GetNDF(), fitf.GetChisquare(), fitf.GetProb())
    return result

def gausstailfit(name, project_hist):
    
    max_bin = project_hist.GetMaximumBin()
    max_value = project_hist.GetBinCenter(max_bin)
    rms_value = project_hist.GetRMS()
    max_y = project_hist.GetMaximum()
#     print max_bin, max_value, rms_value
    def gausstail(x, p):
        #         // [Constant] * ROOT::Math::crystalball_function(x, [Alpha], [N], [Sigma], [Mean])
        return p[0] * ROOT.Math.crystalball_function(x[0], p[3], p[4], p[2], p[1])

    fitf = ROOT.TF1('gausstail', gausstail, -100, 100, 5)
    fitf.SetParNames('norm', 'mean', 'sigma', 'alpha', 'n')
#     fitf.FixParameter(0, 1.)
    fitf.SetParLimits(1, max_value-15.,max_value+15.)

    fitf.SetParameters(max_y, max_value, rms_value, 1, 1)
    draw([project_hist], labels=['fit'], text=name)
    stuff.append(fitf)
    project_hist.Draw("same")
#     c.Draw()
    print '   name: {}, y_max = {}, max_value = {}, RMS = {}'.format(name, max_y, max_value, rms_value)
    result = project_hist.Fit('gausstail','QERLS+')
    result.Print()
    print '   norm = {}, reso_mean = {}, reso_sigma = {}, alpha = {}, n = {}'.format(result.GetParams()[0], result.GetParams()[1], result.GetParams()[2], result.GetParams()[3], result.GetParams()[4])
#     func = project_hist.GetFunction("gaus")
    # print '   NDF = {}, chi2 = {}, prob = {}'.format(fitf.GetNDF(), fitf.GetChisquare(), fitf.GetProb())
    return result

def gausstailfit_pt(name, project_hist):
    
    max_bin = project_hist.GetMaximumBin()
    max_value = project_hist.GetBinCenter(max_bin)
    rms_value = project_hist.GetRMS()
    max_y = project_hist.GetMaximum()
#     print max_bin, max_value, rms_value
    def gausstail(x, p):
        #         // [Constant] * ROOT::Math::crystalball_function(x, [Alpha], [N], [Sigma], [Mean])
        return p[0] * ROOT.Math.crystalball_function(x[0], p[3], p[4], p[2], p[1])

    fitf = ROOT.TF1('gausstail', gausstail, 0, 2, 5)
    fitf.SetParNames('norm', 'mean', 'sigma', 'alpha', 'n')
#     fitf.FixParameter(0, 1.)
    fitf.SetParLimits(0, max_y*0.95,max_y*1.05)

    fitf.SetParLimits(1, max_value*0.95,max_value*1.05)
    fitf.SetParameters(max_y, max_value, 0.05, 1, 1)

#     fitf.SetParameters(max_y, max_value, rms_value, 1, 1)
    draw([project_hist], labels=['fit'], text=name)
    stuff.append(fitf)
    project_hist.Draw("same")
#     c.Draw()
    print '   name: {}, y_max = {}, max_value = {}, RMS = {}'.format(name, max_y, max_value, rms_value)
    result = project_hist.Fit('gausstail','QERLS+')
    result.Print()
    print '   norm = {}, reso_mean = {}, reso_sigma = {}, alpha = {}, n = {}'.format(result.GetParams()[0], result.GetParams()[1], result.GetParams()[2], result.GetParams()[3], result.GetParams()[4])
#     func = project_hist.GetFunction("gaus")
    # print '   NDF = {}, chi2 = {}, prob = {}'.format(fitf.GetNDF(), fitf.GetChisquare(), fitf.GetProb())
    return result


In [ ]:
def computeEResolution(h2d_orig):
    h2d = h2d_orig.Clone()
    h2d.GetYaxis().SetRangeUser(-100, 100)
    x, y, ex_l, ex_h, ey_l, ey_h = [], [], [], [], [], []
    print '-----------------------'
    for x_bin_low, x_bin_high in [(3,6),(7,12),(13,23),(24,34),(35,49),(50,100)]:
        y_proj = h2d.ProjectionY(uuid.uuid4().hex[:6]+'_y', x_bin_low, x_bin_high)
        stuff.append(y_proj)
        x_low = h2d.GetXaxis().GetBinLowEdge(x_bin_low)
        x_high = h2d.GetXaxis().GetBinUpEdge(x_bin_high)
        print 'x_low: {} x_high: {}'.format(x_low, x_high)
        fit_result = gausstailfit(h2d_orig.GetName(), y_proj)

        h2d.SetAxisRange(x_low, x_high)
        x_mean = h2d.GetMean()
#         print 'mean: {}'.format(x_mean)
    #     x_value = h2d.GetXaxis().GetBinCenter(x_bin)
    #     x_err_min = x_value - h2d.GetXaxis().GetBinLowEdge(x_bin)
    #     x_err_plus = h2d.GetXaxis().GetBinUpEdge(x_bin) - x_value

        x.append(x_mean)
        ex_l.append(0)
        ex_h.append(0)

        y.append(fit_result.GetParams()[2]/x_mean)
        ey_l.append(0)
        ey_h.append(0)
    return x, y, ex_l, ex_h, ey_l, ey_h



In [ ]:
def computePtRespResolutionMean(h2d_orig):
    h2d = h2d_orig.Clone()
    h2d.GetYaxis().SetRangeUser(0, 2)
    x, y, ex_l, ex_h, ey_l, ey_h = [], [], [], [], [], []
    print '-----------------------'
    for x_bin_low, x_bin_high in [(3,6),(7,12),(13,23),(24,34),(35,49),(50,100)]:
        y_proj = h2d.ProjectionY(uuid.uuid4().hex[:6]+'_y', x_bin_low, x_bin_high)
        stuff.append(y_proj)
        x_low = h2d.GetXaxis().GetBinLowEdge(x_bin_low)
        x_high = h2d.GetXaxis().GetBinUpEdge(x_bin_high)
        print 'x_low: {} x_high: {}'.format(x_low, x_high)
        fit_result = gausstailfit_pt(h2d_orig.GetName(), y_proj)

        h2d.SetAxisRange(x_low, x_high)
        x_mean = h2d.GetMean()
#         print 'mean: {}'.format(x_mean)
    #     x_value = h2d.GetXaxis().GetBinCenter(x_bin)
    #     x_err_min = x_value - h2d.GetXaxis().GetBinLowEdge(x_bin)
    #     x_err_plus = h2d.GetXaxis().GetBinUpEdge(x_bin) - x_value

        x.append(x_mean)
        ex_l.append(0)
        ex_h.append(0)

        y.append(fit_result.GetParams()[1])
        ey_l.append(0)
        ey_h.append(0)
    return x, y, ex_l, ex_h, ey_l, ey_h


In [ ]:
def computePtRespResolutionTest(h2d_orig):
    h2d = h2d_orig.Clone()
    h2d.GetYaxis().SetRangeUser(0, 2)
    x, y, ex_l, ex_h, ey_l, ey_h = [], [], [], [], [], []
    print '-----------------------'
    for x_bin_low, x_bin_high in [(3,100)]:
        y_proj = h2d.ProjectionY(uuid.uuid4().hex[:6]+'_y', x_bin_low, x_bin_high)
        stuff.append(y_proj)
        x_low = h2d.GetXaxis().GetBinLowEdge(x_bin_low)
        x_high = h2d.GetXaxis().GetBinUpEdge(x_bin_high)
        print 'x_low: {} x_high: {}'.format(x_low, x_high)
        fit_result = gausstailfit_pt(h2d_orig.GetName(), y_proj)

        h2d.SetAxisRange(x_low, x_high)
        x_mean = h2d.GetMean()
#         print 'mean: {}'.format(x_mean)
    #     x_value = h2d.GetXaxis().GetBinCenter(x_bin)
    #     x_err_min = x_value - h2d.GetXaxis().GetBinLowEdge(x_bin)
    #     x_err_plus = h2d.GetXaxis().GetBinUpEdge(x_bin) - x_value

        x.append(x_mean)
        ex_l.append(0)
        ex_h.append(0)

        y.append(fit_result.GetParams()[2])
        ey_l.append(0)
        ey_h.append(0)
    return x, y, ex_l, ex_h, ey_l, ey_h

In [ ]:
def computePtRespResolution(h2d_orig):
    h2d = h2d_orig.Clone()
    h2d.GetYaxis().SetRangeUser(0, 2)
    x, y, ex_l, ex_h, ey_l, ey_h = [], [], [], [], [], []
    print '-----------------------'
    for x_bin_low, x_bin_high in [(3,6),(7,12),(13,23),(24,34),(35,49),(50,100)]:
        y_proj = h2d.ProjectionY(uuid.uuid4().hex[:6]+'_y', x_bin_low, x_bin_high)
        stuff.append(y_proj)
        x_low = h2d.GetXaxis().GetBinLowEdge(x_bin_low)
        x_high = h2d.GetXaxis().GetBinUpEdge(x_bin_high)
        print 'x_low: {} x_high: {}'.format(x_low, x_high)
        fit_result = gausstailfit_pt(h2d_orig.GetName(), y_proj)

        h2d.SetAxisRange(x_low, x_high)
        x_mean = h2d.GetMean()
#         print 'mean: {}'.format(x_mean)
    #     x_value = h2d.GetXaxis().GetBinCenter(x_bin)
    #     x_err_min = x_value - h2d.GetXaxis().GetBinLowEdge(x_bin)
    #     x_err_plus = h2d.GetXaxis().GetBinUpEdge(x_bin) - x_value

        x.append(x_mean)
        ex_l.append(0)
        ex_h.append(0)

        y.append(fit_result.GetParams()[2])
        ey_l.append(0)
        ey_h.append(0)
    return x, y, ex_l, ex_h, ey_l, ey_h


In [ ]:
def computeEResolutionMean(h2d_orig):
    h2d = h2d_orig.Clone()
    h2d.GetYaxis().SetRangeUser(-100, 100)
    x, y, ex_l, ex_h, ey_l, ey_h = [], [], [], [], [], []
    print '-----------------------'
    for x_bin_low, x_bin_high in [(3,6),(7,12),(13,23),(24,34),(35,49),(50,100)]:
        y_proj = h2d.ProjectionY(uuid.uuid4().hex[:6]+'_y', x_bin_low, x_bin_high)
        stuff.append(y_proj)
        x_low = h2d.GetXaxis().GetBinLowEdge(x_bin_low)
        x_high = h2d.GetXaxis().GetBinUpEdge(x_bin_high)
        print 'x_low: {} x_high: {}'.format(x_low, x_high)
        fit_result = gausstailfit(h2d_orig.GetName(), y_proj)

        h2d.SetAxisRange(x_low, x_high)
        x_mean = h2d.GetMean()
#         print 'mean: {}'.format(x_mean)
    #     x_value = h2d.GetXaxis().GetBinCenter(x_bin)
    #     x_err_min = x_value - h2d.GetXaxis().GetBinLowEdge(x_bin)
    #     x_err_plus = h2d.GetXaxis().GetBinUpEdge(x_bin) - x_value

        x.append(x_mean)
        ex_l.append(0)
        ex_h.append(0)

        y.append(fit_result.GetParams()[1])
        ey_l.append(0)
        ey_h.append(0)
    return x, y, ex_l, ex_h, ey_l, ey_h


In [ ]:
# sample='ele-V9'

In [ ]:
for smp in ['ele-V8', 'ele-V9']:
    for tp in tps:
        for pu in ['PU200']:
            for tp_sel in ['EGq3', 'EGq4']:
                hsets, labels, text = hplot.get_histo(histos.EGResoHistos, smp, pu, tp, tp_sel, 'GENEtaBC', debug=False)
                draw([his.h_ptRespVpt_graph('sigmaPtResp', '#sigma_{p_{T}^{L1}/p_{T}^{GEN}}', computePtRespResolution) for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)


In [ ]:
for smp in ['ele-V8', 'ele-V9']:
    for tp in tps:
        for pu in ['PU200']:
            for tp_sel in ['EGq3', 'EGq4']:
                hsets, labels, text = hplot.get_histo(histos.EGResoHistos, smp, pu, tp, tp_sel, 'GENEtaD', debug=False)
                draw([his.h_ptRespVpt_graph('meanPtResp', '<p_{T}^{L1}/p_{T}^{GEN}>', computePtRespResolutionMean) for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)


In [ ]:
hsets, labels, text = hplot.get_histo(histos.EGResoHistos, ['ele-V9'], 'PU200', ['EG'], ['EGq3', 'EGq4'], 'GENEtaBC', debug=False)
draw([his.g_ptRespVpt_sigmaPtResp for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.3)
draw([his.g_ptRespVpt_meanPtResp for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=2)


#### Matching to the highes pT cluster in DR 0.2 from GEN

In [ ]:
# ROOT.gStyle.SetPadTopMargin(0.05)
# ROOT.gStyle.SetPadTopMargin(0.13)
ROOT.gStyle.SetPadBottomMargin(0.13)
ROOT.gStyle.SetPadLeftMargin(0.13)
ROOT.gStyle.SetPadRightMargin(0.30)

ROOT.gStyle.SetCanvasBorderMode(0)
ROOT.gStyle.SetCanvasColor(0)
ROOT.gStyle.SetCanvasDefH(600)
ROOT.gStyle.SetCanvasDefW(800)


## Angular resolution

In [ ]:
sample

In [ ]:

for tp in tps:
    for tp_sel in tp_select[tp]:
        for gen_sel in gen_select[tp]:
            hsets, labels, text = hplot.get_histo(histos.EGResoHistos, sample, ['PU0', 'PU200'], tp, tp_sel, gen_sel, False)            
            draw([his.h_phiRes for his in hsets], labels, norm=True, text=text, options='hist')


In [ ]:

for tp in tps:
    for tp_sel in tp_select[tp]:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.EGResoHistos, sample, ['PU0', 'PU200'], tp, tp_sel, gen_sel)            
            draw([his.h_etaRes for his in hsets], labels, norm=True, text=text, options='hist')


In [ ]:
for tp in tps:
    for tp_sel in tp_select[tp]:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.EGResoHistos, ['ele'], ['PU0', 'PU200'], tp, tp_sel, gen_sel)            
            draw([his.h_drRes for his in hsets], labels, norm=True, text=text)


In [ ]:
hsets, labels, text = hplot.get_histo(histos.EGResoHistos, sample, ['PU200'], 'EG', ['EGq4','EGq5'], 'GENEtaBCPt15', False)            
draw([his.h_phiRes for his in hsets], labels, norm=True, text=text, options='hist', min_x=-0.3, max_x=0.3,
     do_write=True, write_name='egammaEndcap_EG_HwQual4_EtaBC_phiRes')
hsets, labels, text = hplot.get_histo(histos.EGResoHistos, sample, ['PU200'], 'EG', ['EGq4','EGq5'], 'GENEtaBCPt15', False)            
draw([his.h_etaRes for his in hsets], labels, norm=True, text=text, options='hist', min_x=-0.3, max_x=0.3,
     do_write=True, write_name='egammaEndcap_EG_HwQual4_EtaBC_etaRes')


## $p_{T}$ response

In [ ]:
for tp in tps:
    for tp_sel in tp_select[tp]:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.EGResoHistos, sample, ['PU0', 'PU200'], tp, tp_sel, gen_sel)            
            draw([his.h_ptRes for his in hsets], labels, norm=True, text=text, options='hist')


In [ ]:
ROOT.gStyle.SetLegendTextSize(0.002)

In [12]:
hsets, labels, text = hplot.get_histo(histos.EGResoHistos, sample, ['PU200'], 'EG', ['EGq5', 'EGq4'], ['GENEtaBCPt15'])   
text_forprint = 'EG endcap, 1.52 < |#eta^{GEN}| #leq 2.4, p_{T}^{GEN} #geq 15GeV'
labels_forprint = ['w/o brem. recovery', 'w brem. recovery']
legend_position = (0.55, 0.7, 0.8, 0.8)
c1 = draw([his.h_ptRes for his in hsets], labels_forprint, y_axis_label='a.u.', 
          norm=True, text=text_forprint, options='hist', legend_position=legend_position,
          do_write=True, write_name='figs/egammaEndcap_EG_hwqual5_EtaBC_ptRes')


Info in <TCanvas::Print>: pdf file figs/egammaEndcap_EG_hwqual5_EtaBC_ptRes.pdf has been created


In [13]:
hsets, labels, text = hplot.get_histo(histos.EGResoHistos, sample, ['PU200'], 'EG', ['EGq5', 'EGq4'], ['GENEtaDPt15'])   
text_forprint = 'EG endcap, 2.4 < |#eta^{GEN}| #leq 2.8, p_{T}^{GEN} #geq 15GeV'
labels_forprint = ['w/o brem. recovery', 'w brem. recovery']
legend_position = (0.55, 0.7, 0.8, 0.8)
c1 = draw([his.h_ptRes for his in hsets], labels_forprint, y_axis_label='a.u.', 
          norm=True, text=text_forprint, options='hist',  legend_position=legend_position,
          do_write=True, write_name='figs/egammaEndcap_EG_hwqual5_EtaD_ptRes')


Info in <TCanvas::Print>: pdf file figs/egammaEndcap_EG_hwqual5_EtaD_ptRes.pdf has been created


In [ ]:
hsets, labels, text = hplot.get_histo(histos.EGResoHistos, sample, ['PU200'], 'EG', ['EGq2', 'EGq3', 'EGq4', 'EGq5'], 'GENEtaBC')   
c1 = draw([his.h_ptResp for his in hsets], labels, y_axis_label='a.u.', norm=True, text=text, options='hist', do_write=True, write_name='egammaEndcap_EG_hwqual4_EtaBC_ptResp')


In [ ]:
hsets, labels, text = hplot.get_histo(histos.EGResoHistos, sample, ['PU200'], 'EG', ['EGq2', 'EGq3', 'EGq4', 'EGq5'], 'GENEtaD')   
c1 = draw([his.h_ptRes for his in hsets], labels, y_axis_label='a.u.', norm=True, text=text, options='hist', do_write=True, write_name='egammaEndcap_EG_hwqual4_EtaBC_ptResp')


In [ ]:
hsets, labels, text = hplot.get_histo(histos.EGResoHistos, sample, ['PU200'], 'TkEleEL', ['EGq2', 'EGq3', 'EGq4', 'EGq5'], 'GENEtaBC')   
c1 = draw([his.h_ptResp for his in hsets], labels, y_axis_label='a.u.', norm=True, text=text, options='hist', do_write=False, write_name='egammaEndcap_EG_hwqual4_EtaBC_ptResp')


In [ ]:
sample='ele-V9'

In [ ]:
hsets, labels, text = hplot.get_histo(histos.EGResoHistos, sample, ['PU200'], 'EG', 'EGq4', 'GENEtaBC')   
c1 = draw([his.h_ptResp for his in hsets], labels, y_axis_label='a.u.', norm=True, text=text, options='hist', do_write=True, write_name='egammaEndcap_EG_hwqual4_EtaBC_ptResp')
hsets, labels, text = hplot.get_histo(histos.EGResoHistos, sample, ['PU200'], 'EG', 'EGq4', 'GENEtaD')            
c2 = draw([his.h_ptResp for his in hsets], labels, y_axis_label='a.u.', norm=True, text=text, options='hist', do_write=True, write_name='egammaEndcap_EG_hwqual4_EtaD_ptResp')


In [ ]:
SaveCanvas(c1, 'EG_hwqual4_EtaBC_ptResp')
SaveCanvas(c2, 'EG_hwqual4_EtaD_ptResp')


In [ ]:

for tp in tps:
    for tp_sel in tp_select[tp]:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.EGResoHistos, sample, ['PU0', 'PU200'], tp, tp_sel, gen_sel)            
            draw([his.h_ptResp for his in hsets], labels, norm=True, text=text, options='hist')


In [ ]:
for tp in tps:
#     for tp_sel in tp_select[tp]:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.EGResoHistos, sample, ['PU200'], tp, ['EGq4', 'EGq4Pt20'], gen_sel)            
            draw([his.h_ptResp for his in hsets], labels, norm=True, text=text, options='hist')


In [ ]:

for tp in tps:
    for pu in ['PU0', 'PU200']:
        for gen_sel in ['GENEtaBC', 'GENEtaBCLBrem',  'GENEtaBCHBrem']:
            hsets, labels, text = hplot.get_histo(histos.EGResoHistos, sample, pu, tp, ['EGq3', 'EGq4'], gen_sel)            
            draw([his.h_ptResp for his in hsets], labels, norm=True, text=text, options='hist')


In [ ]:
for tp in tps:
    for pu in ['PU0', 'PU200']:
        for gen_sel in ['GENEtaBC', 'GENEtaBCLBrem',  'GENEtaBCHBrem']:
            hsets, labels, text = hplot.get_histo(histos.EGResoHistos, sample, pu, tp, ['EGq3Pt30', 'EGq4Pt30'], gen_sel)            
            draw([his.h_ptResp for his in hsets], labels, norm=True, text=text, options='hist')


In [ ]:

for tp in tps:
    for pu in ['PU0', 'PU200']:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.EGResoHistos, sample, pu, tp, ['EGq2', 'EGq3'], gen_sel)            
            draw([his.h_ptResp for his in hsets], labels, norm=True, text=text, options='hist')


In [ ]:
for tp in tps:
    for tp_sel in tp_select:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.EGResoHistos, ['ele'], ['PU0', 'PU200'], tp, tp_sel, gen_sel)            
            draw([his.h_ptRespVpt for his in hsets], labels, options="COLZ", do_profile=False, text=text)


In [ ]:
for tp in tps:
    for tp_sel in tp_select:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.EGResoHistos, ['ele'], ['PU0', 'PU200'], tp, tp_sel, gen_sel)            
            draw([his.h_ptRespVpt_mdn() for his in hsets], labels, options="COLZ", do_profile=False, text=text, min_y=0, max_y=3)


In [ ]:
for tp in tps:
    for tp_sel in tp_select:
        for gen_sel in ['GENEtaC']:
            hsets, labels, text = hplot.get_histo(histos.EGResoHistos, ['ele'], ['PU0', 'PU200'], tp, tp_sel, gen_sel)            
            draw([his.h_ptRespVpt_gmean() for his in hsets], labels, options="COLZ", do_profile=False, text=text, min_y=0, max_y=3)


In [ ]:
for tp in tps:
    for pu in ['PU0', 'PU200']:
    #for tp_sel in tp_select:
        for gen_sel in ['GENEtaC']:
            hsets, labels, text = hplot.get_histo(histos.EGResoHistos, ['ele'], pu, tp, ['EGq2', 'EGq3'], gen_sel)            
            draw([his.h_ptRespVpt_sigma() for his in hsets], labels, options="COLZ", do_profile=False, text=text, min_y=0, max_y=3)


In [ ]:
for tp in tps:
    for tp_sel in tp_select:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.EGResoHistos, ['ele'], ['PU200'], tp, tp_sel, gen_sel)            
            plots = []
            plots.append(hsets[0].h_ptRespVpt.ProjectionY(uuid.uuid4().hex[:6], 21, 22))
            plots.append(hsets[0].h_ptRespVpt.ProjectionY(uuid.uuid4().hex[:6], 22, 23))
            drawSame(plots, ['A', 'B'], text=text)
            #drawSeveral([his.h_ptRespVpt for his in hsets], labels, options="COLZ", do_profile=False, text=text)


In [ ]:
for tp in tps:
    for tp_sel in tp_select:
        for gen_sel in ['GEN']:
            hsets, labels, text = hplot.get_histo(histos.EGResoHistos, 'ele', ['PU0', 'PU200'], tp, tp_sel, gen_sel)            
            print labels
            print text
            drawSeveral([his.h_ptRespVeta for his in hsets], labels, options="COLZ", do_profile=False, text=text)


#### All clusters in DR 0.2 from GEN

In [ ]:
histo_resoCone_df = pd.DataFrame(index=tp_sets, columns=particles)
for tp_set in tp_sets:
    for particle in particles:
        histo_resoCone_df.loc[tp_set][particle] =  [histos.HistoSetReso('{}_{}'.format(tp_set, particle), sample.histo_file).hresoCone for sample in samples]
histo_resoCone_df

In [ ]:
for particle in particles:
    for tp_set in ['DEF', 'DEF_em']:
        hByTPset = [histo.h_ptRes for histo in histo_resoCone_df.loc[tp_set, particle]]
        drawSame(hByTPset, labels, norm=True, text=titles_df.loc[tp_set, particle])


In [ ]:
for particle in particles:
    for tp_set in ['DEF', 'DEF_em']:
        hByTPset = [histo.h_ptResp for histo in histo_resoCone_df.loc[tp_set, particle]]
        drawSame(hByTPset, labels, norm=True, text=titles_df.loc[tp_set, particle])


In [ ]:
for particle in particles:
    for tp_set in ['DEF', 'DEF_em']:
        hByTPset = [histo.h_n010 for histo in histo_resoCone_df.loc[tp_set, particle]]
        drawSame(hByTPset, labels, norm=True, text=titles_df.loc[tp_set, particle])


In [ ]:
for particle in particles:
    for tp_set in ['DEF', 'DEF_em']:
        hByTPset = [histo.h_n025 for histo in histo_resoCone_df.loc[tp_set, particle]]
        drawSame(hByTPset, labels, norm=True, text=titles_df.loc[tp_set, particle])


We now compare the 2 defitnitions (highest pt and all clusters in cone) for the same PU point

In [ ]:
for particle in particles:
    for tp_set in ['DEF', 'DEF_em']:
        hByTPset_best = [histo.h_ptRes for histo in histo_reso_df.loc[tp_set, particle]]
        hByTPset_cone = [histo.h_ptRes for histo in histo_resoCone_df.loc[tp_set, particle]]

        drawSame([hByTPset_best[0], hByTPset_cone[0]], ['PU0 best', 'PU0 cone'], norm=True, text=titles_df.loc[tp_set, particle])
        drawSame([hByTPset_best[1], hByTPset_cone[1]], ['PU200 best', 'PU200 cone'], norm=True, text=titles_df.loc[tp_set, particle])


We now show the behavior of the pt residuals vs pt of the GEN particle

In [ ]:
for algo in algos:
    hByAlgo = [histo.h_ptResVpt for histo in histosAlgos_all.loc[algo].hresoCone]
    #normalize(hclAll_pt35_layer, nevents_pt35)
    drawSeveral(hByAlgo, histosAlgos_all.loc[algo]['labels'], 'COLZ',do_profile=True)
    #print hByAlgo[0].GetName()

In [ ]:
for algo in algos:
    hCl3D_ptResVpt_0 = [histo.h_ptResVpt.ProjectionY(uuid.uuid4().hex[:6], 1, 5) for histo in histosAlgos_all.loc[algo].hresoCone]
    for histo in hCl3D_ptResVpt_0:
        histo.SetTitle('3D cluster reso (GeV) for 1 < pt < 10 GeV')
    #     normalize(hByAlgo, histosAlgos_all.loc[algo]['norm'])
    drawSame(hCl3D_ptResVpt_0, histosAlgos_all.loc[algo]['labels'], norm=True)


In [ ]:
for algo in algos:
    hCl3D_ptResVpt_1 = [histo.h_ptResVpt.ProjectionY(uuid.uuid4().hex[:6], 6, 10) for histo in histosAlgos_all.loc[algo].hresoCone]
    for histo in hCl3D_ptResVpt_1:
        histo.SetTitle('3D cluster reso (GeV) for 10 < pt < 20 GeV')
    #     normalize(hByAlgo, histosAlgos_all.loc[algo]['norm'])
    drawSame(hCl3D_ptResVpt_1, histosAlgos_all.loc[algo]['labels'], norm=True)


In [ ]:
for algo in algos:
    hCl3D_ptResVpt_2 = [histo.h_ptResVpt.ProjectionY(uuid.uuid4().hex[:6], 11, 15) for histo in histosAlgos_all.loc[algo].hresoCone]
    for histo in hCl3D_ptResVpt_2:
        histo.SetTitle('3D cluster reso (GeV) for 20 < pt < 30 GeV')
    #     normalize(hByAlgo, histosAlgos_all.loc[algo]['norm'])
    drawSame(hCl3D_ptResVpt_2, histosAlgos_all.loc[algo]['labels'], norm=True)


In [ ]:
for algo in algos:
    hCl3D_ptResVpt_3 = [histo.h_ptResVpt.ProjectionY(uuid.uuid4().hex[:6], 16, 20) for histo in histosAlgos_all.loc[algo].hresoCone]
    for histo in hCl3D_ptResVpt_3:
        histo.SetTitle('3D cluster reso (GeV) for 30 < pt < 40 GeV')
    #     normalize(hByAlgo, histosAlgos_all.loc[algo]['norm'])
    drawSame(hCl3D_ptResVpt_3, histosAlgos_all.loc[algo]['labels'], norm=True)


In [ ]:
for algo in algos:
    hByAlgo = [histo.h_ptResVeta for histo in histosAlgos_all.loc[algo].hresoCone]
    #normalize(hclAll_pt35_layer, nevents_pt35)
    drawSeveral(hByAlgo, histosAlgos_all.loc[algo]['labels'], 'COLZ',do_profile=True)

In [ ]:
for algo in algos:
    hByAlgo = [histo.h_ptResVnclu for histo in histosAlgos_all.loc[algo].hresoCone]
    #normalize(hclAll_pt35_layer, nevents_pt35)
    drawSeveral(hByAlgo, histosAlgos_all.loc[algo]['labels'], 'COLZ',do_profile=True)

### Energy resolution


In [ ]:


for algo in algos:
    hByAlgo = [histo.h_energyRes for histo in histosAlgos_all.loc[algo].hreso]
    #normalize(hclAll_pt35_layer, nevents_pt35)
    drawSame(hByAlgo, histosAlgos_all.loc[algo]['labels'], norm=True)



In [ ]:

for s_id in range(0, len(samples)):
    labels_bySample = histosAlgos_all['labels'].apply(lambda x: x[s_id])
    hBySample = [histo.h_energyRes for histo in  histosAlgos_all['hreso'].apply(lambda x: x[s_id])]
    drawSame(hBySample, labels_bySample,'hist',logy=True)


## Postion resolution of 2D clusters

In [ ]:
for algo in algos:
    hByAlgo = [histo.h_xResVlayer for histo in histosAlgos_all.loc[algo].hreso2D]
    #normalize(hclAll_pt35_layer, nevents_pt35)
    drawSeveral(hByAlgo, histosAlgos_all.loc[algo]['labels'], 'COLZ',do_profile=True, miny=-2, maxy=2)

In [ ]:
for algo in algos:
    hByAlgo = [histo.h_xResVlayer.ProjectionY(uuid.uuid4().hex[:6], 1, 3) for histo in histosAlgos_all.loc[algo].hreso2D]
    for histo in hByAlgo:
        histo.SetTitle('2D cluster x reso (cm) for layer 1 and 3')
    #     normalize(hByAlgo, histosAlgos_all.loc[algo]['norm'])
    drawSame(hByAlgo, histosAlgos_all.loc[algo]['labels'], norm=True, options='hist')


In [ ]:
for algo in algos:
    hByAlgo = [histo.h_xResVlayer.ProjectionY(uuid.uuid4().hex[:6], 12, 15) for histo in histosAlgos_all.loc[algo].hreso2D]
    for histo in hByAlgo:
        histo.SetTitle('2D cluster x reso (cm) for layer 13 and 15')
    #     normalize(hByAlgo, histosAlgos_all.loc[algo]['norm'])
    drawSame(hByAlgo, histosAlgos_all.loc[algo]['labels'], norm=True, options='hist')


In [ ]:
for algo in algos:
    hByAlgo = [histo.h_yResVlayer for histo in histosAlgos_all.loc[algo].hreso2D]
    #normalize(hclAll_pt35_layer, nevents_pt35)
    drawSeveral(hByAlgo, histosAlgos_all.loc[algo]['labels'], 'COLZ',do_profile=True, miny=-2, maxy=2)